In [1]:
import streamlit as st

import pandas as pd
import geopandas as gpd

import plotly.express as px

import folium
from folium import Marker
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from streamlit_folium import folium_static

In [16]:
registros = pd.read_csv("felinos.csv", sep="/t")

C:\Users\XPC\AppData\Local\Temp\ipykernel_19648\1583165765.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  registros = pd.read_csv("felinos.csv", sep="/t")


In [22]:
registros.insert(1, 'count', 1)

In [24]:
registros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 2 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                            

In [25]:
  can = gpd.read_file("cantones/cantones.geojson")

In [28]:
# Limpieza de datos
# Eliminación de registros con valores nulos en la columna 'species'
registros = registros[registros['species'].notna()]
# Cambio del tipo de datos del campo de fecha
registros["eventDate"] = pd.to_datetime(registros["eventDate"])

# Especificación de filtros
# Especie
lista_especies = registros.species.unique().tolist()
lista_especies.sort()
filtro_especie = st.sidebar.selectbox('Seleccione la especie', lista_especies)


#
# PROCESAMIENTO
#

# Filtrado
registros = registros[registros['species'] == filtro_especie]

# Cálculo de la cantidad de registros en los cantones
# "Join" espacial de las capas de cantones y registros de presencia de especies
can_contienen_registros = can.sjoin(registros, how="left", predicate="contains")
# Conteo de registros de presencia en cada provincia
can_registros = can_contienen_registros.groupby("cod_provin").agg(cantidad_registros = ("gbifID","count"))
can_registros = can_registros.reset_index() # para convertir la serie a dataframe

KeyError: 'species'